# EHDSLens Visualization

This notebook demonstrates how to create visualizations from the EHDS systematic review data using EHDSLens.

## Requirements

```bash
pip install ehdslens[viz]
# or
pip install matplotlib plotly
```

In [ ]:
from ehdslens import EHDSAnalyzer
from ehdslens.visualization import EHDSVisualizer
from ehdslens.data import ThematicAxis, QualityRating

# Load data
analyzer = EHDSAnalyzer()
analyzer.load_default_data()

# Create visualizer
viz = EHDSVisualizer(analyzer.db)

print("Visualizer ready!")

## PRISMA Flow Diagram Data

Get the data needed to create a PRISMA 2020 flow diagram:

In [ ]:
# Get PRISMA diagram data
prisma = viz.create_prisma_diagram_data()

print("=" * 60)
print("PRISMA 2020 FLOW DIAGRAM DATA")
print("=" * 60)

print("\n📥 IDENTIFICATION")
print(f"   Records from databases: {prisma['identification']['total_database']}")
print(f"   - Web of Science: {prisma['identification']['web_of_science']}")
print(f"   - Scopus: {prisma['identification']['scopus']}")
print(f"   - PubMed: {prisma['identification']['pubmed']}")
print(f"   - IEEE Xplore: {prisma['identification']['ieee']}")
print(f"   Grey literature: {prisma['identification']['grey_literature']}")

print("\n🔍 SCREENING")
print(f"   Duplicates removed: {prisma['screening']['duplicates_removed']}")
print(f"   Records screened: {prisma['screening']['records_screened']}")
print(f"   Records excluded: {prisma['screening']['records_excluded']}")

print("\n📋 ELIGIBILITY")
print(f"   Full-text assessed: {prisma['eligibility']['full_text_assessed']}")
print(f"   Excluded: {prisma['eligibility']['full_text_excluded']}")

print("\n✅ INCLUDED")
print(f"   Total included: {prisma['included']['total']}")
print(f"   - Peer-reviewed: {prisma['included']['peer_reviewed']}")
print(f"   - Grey literature: {prisma['included']['grey_literature']}")

## ASCII Charts (Terminal-friendly)

In [ ]:
# Year distribution ASCII chart
print(viz.create_year_distribution_chart(format="ascii"))

In [ ]:
# Axis distribution ASCII chart
print(viz.create_axis_distribution_chart(format="ascii"))

In [ ]:
# Quality distribution ASCII chart
print(viz.create_quality_distribution_chart(format="ascii"))

## Matplotlib Visualizations

Create publication-quality charts using Matplotlib:

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Get chart data
year_data = viz.create_year_distribution_chart(format="data")

# Create figure
fig, ax = plt.subplots(figsize=(10, 5))

years = list(year_data.keys())
counts = list(year_data.values())

bars = ax.bar(years, counts, color='steelblue', edgecolor='white')
ax.set_xlabel('Publication Year', fontsize=12)
ax.set_ylabel('Number of Studies', fontsize=12)
ax.set_title('EHDS Systematic Review: Publication Year Distribution', fontsize=14, fontweight='bold')

# Add value labels on bars
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
            str(count), ha='center', va='bottom', fontsize=10)

ax.set_xticks(years)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Thematic Axis Distribution
axis_data = viz.create_axis_distribution_chart(format="data")

fig, ax = plt.subplots(figsize=(12, 6))

# Shorten axis names for display
short_names = {
    'governance_rights_ethics': 'Governance & Ethics',
    'secondary_use_pets': 'Secondary Use & PETs',
    'national_implementation': 'National Implementation',
    'citizen_engagement': 'Citizen Engagement',
    'federated_learning_ai': 'FL & AI'
}

axes = [short_names.get(k, k) for k in axis_data.keys()]
counts = list(axis_data.values())

colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B1F2B']

bars = ax.barh(axes, counts, color=colors, edgecolor='white')
ax.set_xlabel('Number of Studies', fontsize=12)
ax.set_title('Studies by Thematic Axis', fontsize=14, fontweight='bold')

# Add value labels
for bar, count in zip(bars, counts):
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
            str(count), ha='left', va='center', fontsize=11)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Quality Distribution Pie Chart
quality_data = viz.create_quality_distribution_chart(format="data")

fig, ax = plt.subplots(figsize=(8, 8))

labels = ['High', 'Moderate', 'Low', 'N/A']
sizes = [quality_data.get('high', 0), quality_data.get('moderate', 0), 
         quality_data.get('low', 0), quality_data.get('not_applicable', 0)]
colors = ['#2E7D32', '#FBC02D', '#E65100', '#9E9E9E']
explode = (0.05, 0, 0, 0)

wedges, texts, autotexts = ax.pie(sizes, explode=explode, labels=labels, colors=colors,
                                   autopct='%1.1f%%', shadow=False, startangle=90,
                                   textprops={'fontsize': 12})

ax.set_title('Quality Rating Distribution (MMAT)', fontsize=14, fontweight='bold')

# Add legend
ax.legend(wedges, [f'{l}: {s} studies' for l, s in zip(labels, sizes)],
          title="Quality Ratings", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
plt.show()

## Plotly Interactive Charts

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Year distribution - interactive
year_data = viz.create_year_distribution_chart(format="data")

fig = px.bar(
    x=list(year_data.keys()),
    y=list(year_data.values()),
    labels={'x': 'Year', 'y': 'Number of Studies'},
    title='EHDS Studies by Publication Year',
    color=list(year_data.values()),
    color_continuous_scale='Blues'
)

fig.update_layout(
    showlegend=False,
    xaxis_tickmode='linear',
    coloraxis_showscale=False
)

fig.show()

In [ ]:
# Create a comprehensive dashboard
db = analyzer.db

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Publications by Year', 'Studies by Thematic Axis',
                    'Quality Distribution', 'Study Types'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'pie'}, {'type': 'pie'}]]
)

# Year distribution
year_data = viz.create_year_distribution_chart(format="data")
fig.add_trace(
    go.Bar(x=list(year_data.keys()), y=list(year_data.values()), 
           marker_color='steelblue', name='Studies'),
    row=1, col=1
)

# Axis distribution
axis_data = viz.create_axis_distribution_chart(format="data")
short_names = ['Governance', 'PETs', 'Implementation', 'Engagement', 'FL/AI']
fig.add_trace(
    go.Bar(x=short_names, y=list(axis_data.values()),
           marker_color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B1F2B'],
           name='Axis'),
    row=1, col=2
)

# Quality pie
quality_data = viz.create_quality_distribution_chart(format="data")
fig.add_trace(
    go.Pie(labels=['High', 'Moderate', 'Low', 'N/A'],
           values=[quality_data.get('high', 0), quality_data.get('moderate', 0),
                   quality_data.get('low', 0), quality_data.get('not_applicable', 0)],
           marker_colors=['#2E7D32', '#FBC02D', '#E65100', '#9E9E9E']),
    row=2, col=1
)

# Study types pie
from collections import Counter
type_counts = Counter(s.study_type.value for s in db.studies)
fig.add_trace(
    go.Pie(labels=list(type_counts.keys()),
           values=list(type_counts.values())),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text='EHDS Systematic Review Dashboard',
    showlegend=False
)

fig.show()

## Export for External Tools

In [ ]:
# Export chart data for Plotly (JSON format)
import json

export_data = {
    "year_distribution": viz.create_year_distribution_chart(format="data"),
    "axis_distribution": viz.create_axis_distribution_chart(format="data"),
    "quality_distribution": viz.create_quality_distribution_chart(format="data"),
    "prisma": viz.create_prisma_diagram_data()
}

print("Exportable chart data:")
print(json.dumps(export_data, indent=2))

In [ ]:
# Save to file
# viz.export_for_plotly(Path("chart_data.json"))
print("\nTo save: viz.export_for_plotly(Path('chart_data.json'))")

## LaTeX/TikZ Export

Generate data formatted for LaTeX documents:

In [ ]:
# LaTeX table format
print("% LaTeX table: Studies by Thematic Axis")
print(r"\begin{tabular}{lc}")
print(r"\toprule")
print(r"Thematic Axis & Studies \\")
print(r"\midrule")

axis_names = {
    'governance_rights_ethics': 'Governance, Rights \\& Ethics',
    'secondary_use_pets': 'Secondary Use \\& PETs',
    'national_implementation': 'National Implementation',
    'citizen_engagement': 'Citizen Engagement',
    'federated_learning_ai': 'Federated Learning \\& AI'
}

for axis, count in axis_data.items():
    print(f"{axis_names.get(axis, axis)} & {count} \\\\")

print(r"\bottomrule")
print(r"\end{tabular}")